In [1]:
import re
import csv
import pandas as pd
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from tqdm import tqdm
import pickle

In [2]:
os.getcwd()

'C:\\Users\\Asus\\Desktop\\Project\\ImageRecommender'

In [285]:
# Counter for unique image names
#image_counter = 0

# we should make another generator to rename the images, so that only the rename generator run once, and then we can run the generator files to get the filepath multiple times

# creating the generator
def reader(filepath):
    current_path = filepath 
    print(current_path)
    
    #global image_counter  
    
    for foldername in os.listdir(current_path):
        folderpath = os.path.join(filepath, foldername)
        current_path = folderpath
        
        if os.path.isdir(folderpath) and not foldername.startswith('.'):
            os.chdir(folderpath)
            print(folderpath)
            
            for file in os.listdir(folderpath):
                if file.endswith(".jpg") or file.endswith(".png"): 
                    with open(os.path.join(folderpath, file), "rb") as f:
                        image = f.read()
                        img = cv2.imread(file)
                    
                    # check if image can be loaded correctly
                    if img is not None:
    
                        # create new name
                        #if file.endswith(".jpg"):
                        #    new_file_name = f"{image_counter}_.jpg"
                        
                        #else:
                        #    new_file_name = f"{image_counter}_.png"
                        
                        #os.rename(file, new_file_name)
                        
                        #image_counter += 1
                        
                        # yield tuple, so later we need to use
                        yield (img, os.path.join(folderpath, file))
                        
                    else:
                        # check path or image format
                        print(f"Error while loading image: {file}")
            
            os.chdir('..')

In [300]:
def get_colour_hist(image):
    
    red_channel = image[:, :, 0]
    green_channel = image[:, :, 1]
    blue_channel = image[:, :, 2]

    # Calculate colour histogram
    red_hist = cv2.calcHist([red_channel], [0], None, [30], [0, 256])
    green_hist = cv2.calcHist([green_channel], [0], None, [30], [0, 256])
    blue_hist = cv2.calcHist([blue_channel], [0], None, [30], [0, 256])

    # Normalize the colour histogram value
    red_hist = cv2.normalize(red_hist, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    green_hist = cv2.normalize(green_hist, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    blue_hist = cv2.normalize(blue_hist, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

    return red_hist, green_hist, blue_hist

In [293]:
filepath = os.path.join(os.getcwd(), "weather_image_recognition")
gen = reader(filepath)

In [294]:
# change to `filepath` for creating the csv there
os.chdir(filepath)

In [295]:
gen = reader(filepath)

master_df = pd.DataFrame()
# LOOP OVER ALL IMAGES

for image, file_path in tqdm(gen, desc="extracting RGB information from the images"):
    file_name = os.path.basename(file_path).replace(".jpg", "").replace(".png", "") # extract the filename
    rgb_array = get_colour_hist(image)
    
    # extract and format the color values
    red_channel = rgb_array[0]
    green_channel = rgb_array[1]
    blue_channel = rgb_array[2]
   
    # merged the list from RGB channel into one list
    merged_list = np.concatenate((red_channel, green_channel, blue_channel), axis=1).flatten().tolist()
    
    # create a DataFrame, tranpose it, add id and merge to master_df
    df = pd.DataFrame(merged_list)
    df = df.transpose()
    df["image_id"] = file_name
    df = df.set_index('image_id')
    
    master_df = pd.concat([master_df, df])
    
master_df.to_csv('color_histogram.csv')

extracting information from the images + renaming by unique ID's: 12it [00:00, 104.73it/s]

C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition
C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition\dew


extracting information from the images + renaming by unique ID's: 1407it [00:14, 94.70it/s] 

C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition\fogsmog


extracting information from the images + renaming by unique ID's: 3101it [00:27, 98.41it/s] 

...

extracting information from the images + renaming by unique ID's: 11114it [01:41, 95.45it/s] 

C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition\sandstorm


extracting information from the images + renaming by unique ID's: 12491it [01:54, 111.88it/s]

C:\Users\Asus\Desktop\Project\ImageRecommender\weather_image_recognition\snow
Error while loading image: 1187.jpg


extracting information from the images + renaming by unique ID's: 13720it [02:06, 108.34it/s]


### check csv

In [296]:
data = pd.read_csv('color_histogram.csv', index_col='image_id')
data.head()

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
image_id,,,,,,,,,,,,,,,,,,,,,
0_,1.000000e+00,8.741844e-02,7.034527e-01,0.011057,0.052840,0.601915,0.003207,0.083466,0.649100,0.002313,...,0.000042,1.295889e-05,0.001407,-1.437002e-10,0.000104,5.275069e-11,0.001708,1.584224e-03,2.844281e-02,2.160399e-02
1000_,-3.637979e-12,0.000000e+00,1.804543e-01,0.000839,0.000061,0.552615,0.002516,0.000245,0.772869,0.006948,...,0.893103,1.644254e-02,0.018962,1.000000e+00,0.026745,3.200172e-02,0.364451,1.000000e+00,9.999999e-01,-2.182787e-10
1001_,2.404346e-03,0.000000e+00,-2.910383e-11,0.344191,0.000829,0.124428,0.788487,0.007918,0.812184,1.000000,...,0.002182,1.618309e-04,0.005431,1.654240e-03,0.000971,6.591220e-03,0.002661,6.311409e-03,1.517224e-02,7.551965e-03
1002_,2.106366e-01,-2.764864e-10,3.567204e-02,1.000000,0.056267,0.733825,0.653571,0.105277,1.000000,0.588742,...,0.001001,1.746230e-10,0.014857,6.839400e-10,0.005823,1.904030e-02,0.006734,1.145186e-01,1.314037e-01,1.133861e-01
1003_,1.000000e+00,2.343813e-03,-2.182787e-11,0.339824,0.009779,0.001660,0.288812,0.020044,0.005371,0.468580,...,0.032910,6.154142e-03,0.001374,2.109375e-02,0.001635,5.576659e-03,0.024609,1.818989e-11,8.731149e-11,3.544922e-02


In [301]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13720 entries, 0_ to 13719_
Data columns (total 90 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       13720 non-null  float64
 1   1       13720 non-null  float64
 2   2       13720 non-null  float64
 3   3       13720 non-null  float64
 4   4       13720 non-null  float64
 5   5       13720 non-null  float64
 6   6       13720 non-null  float64
 7   7       13720 non-null  float64
 8   8       13720 non-null  float64
 9   9       13720 non-null  float64
 10  10      13720 non-null  float64
 11  11      13720 non-null  float64
 12  12      13720 non-null  float64
 13  13      13720 non-null  float64
 14  14      13720 non-null  float64
 15  15      13720 non-null  float64
 16  16      13720 non-null  float64
 17  17      13720 non-null  float64
 18  18      13720 non-null  float64
 19  19      13720 non-null  float64
 20  20      13720 non-null  float64
 21  21      13720 non-null  float64
 22  2

In [303]:
data.loc['1187_']

0     1.000000e+00
1     1.000000e+00
2     5.878974e-09
3     3.283972e-01
4     7.745188e-02
          ...     
85    9.945388e-04
86    1.571559e-01
87    2.685491e-03
88    5.286759e-03
89    4.612580e-01
Name: 1187_, Length: 90, dtype: float64